In [15]:

import pandas as pd
import numpy as np
import glob, time, os, cv2
#%env CUDA_DEVICE_ORDER=PCI_BUS_ID
#%env CUDA_VISIBLE_DEVICES=2

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import shutil
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import Model, optimizers, layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.layers import ZeroPadding2D, AveragePooling2D, Add, BatchNormalization, Conv2D, Dense, Activation, Flatten, Dropout, SpatialDropout2D, MaxPooling2D, GlobalMaxPooling2D, Input
from tensorflow.keras.models import Sequential, Model, model_from_json, model_from_yaml
from tensorflow.keras.metrics import binary_crossentropy
from tensorflow.keras.regularizers import l2, l1
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import preprocess_input
from tensorflow.keras.initializers import glorot_uniform
import tensorflow.keras.backend as K

#from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder 

import matplotlib.pyplot as plt

df_path = '/home/Erdal.Genc/covid_work/deneme/Data_Entry_2017.csv'
img_path = '/mnt/dsets/ChestXrays/NIH/images'
img_path_new= '/home/Erdal.Genc/covid_work/deneme'

print(tf.compat.v1.test.is_gpu_available)
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

img_height=224 #299
img_width=224 #299

<function is_gpu_available at 0x7f4f5b51ef80>
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16497103900606470764
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 10236600826188996969
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10812430746
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15924398540842628232
physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:04:00.0, compute capability: 7.5"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 16876789189805311700
physical_device_desc: "device: XLA_GPU device"
]


In [7]:
import csv

def create_label_directories(csv_filename, img_path):
    directories = set()
    img_path_new= '/home/Erdal.Genc/covid_work/deneme'
    with open(csv_filename, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(csvfile) # skip header row
        for row in reader:
            img_filename = str(row[0])
            labels = str(row[1])
            for label in labels.split('|'):
                #if label == "Hernia":
                    #continue
                src_file = os.path.join(img_path,img_filename)
                label = "_".join(label.split())
                dst_train_dir = os.path.join(img_path_new,"train",label)
                dst_train_file = os.path.join(dst_train_dir,img_filename)
                dst_valid_dir = os.path.join(img_path_new,"valid",label)
                dst_test_dir = os.path.join(img_path_new,"test",label)
                try:
                    if not os.path.exists(dst_train_dir):
                        os.makedirs(dst_train_dir)
                        directories.add(label)
                    if not os.path.exists(dst_valid_dir):
                        os.makedirs(dst_valid_dir)
                    if not os.path.exists(dst_test_dir):
                        os.makedirs(dst_test_dir)
                    src_file_abs = os.path.join(os.getcwd(),src_file)
                    dst_train_file_abs = os.path.join(os.getcwd(),dst_train_file)
                    #print("copy: " + src_file_abs + " to: " + dst_train_file_abs)
                    os.symlink(src_file_abs, dst_train_file_abs)
                except FileNotFoundError:
                    print("FileNotFoundError: " + src_file)
                else:
                    None
    return list(directories)

print(img_path)
directories = create_label_directories(df_path, img_path)
print(directories)

/mnt/dsets/ChestXrays/NIH/images
['Consolidation', 'Hernia', 'No_Finding', 'Nodule', 'Atelectasis', 'Pneumothorax', 'Pleural_Thickening', 'Infiltration', 'Fibrosis', 'Pneumonia', 'Emphysema', 'Mass', 'Effusion', 'Edema', 'Cardiomegaly']


In [8]:
def get_per_label_count(directories):
    per_label_count = []
    img_path_new= '/home/Erdal.Genc/covid_work/deneme'
    for ii in range(len(directories)):
        path, dirs, files = os.walk(os.path.join(img_path_new,"train",directories[ii])).__next__()
        file_count = len(files)
        per_label_count.append(file_count)
    return per_label_count
        
print(directories)
per_label_count = get_per_label_count(directories) 
print(per_label_count)

['Consolidation', 'Hernia', 'No_Finding', 'Nodule', 'Atelectasis', 'Pneumothorax', 'Pleural_Thickening', 'Infiltration', 'Fibrosis', 'Pneumonia', 'Emphysema', 'Mass', 'Effusion', 'Edema', 'Cardiomegaly']
[4667, 227, 60361, 6331, 11559, 5302, 3385, 19894, 1686, 1431, 2516, 5782, 13317, 2303, 2776]


In [9]:
def split_train_valid_test(directories, per_label_count, valid_pct, test_pct):
    img_path_new= '/home/Erdal.Genc/covid_work/deneme'
    for ii in range(len(directories)):
        all_img_paths = glob.glob(os.path.join(img_path_new,"train",directories[ii],"*.*"))
        np.random.shuffle(all_img_paths)
        label_count = per_label_count[ii]
        valid_count = int(label_count*valid_pct)
        valid_files = all_img_paths[:valid_count]
        all_img_paths[:valid_count] = []
        test_count = int(label_count*test_pct)
        test_files = all_img_paths[:test_count]
        all_img_paths[:test_count] = []
        #print(len(valid_files))
        #print(len(test_files))
        train_files = all_img_paths
        all_img_paths = []
        #print(len(train_files))
        for valid_file in valid_files:
            valid_file_abs = os.path.join(os.getcwd(),valid_file)
            #print("move: '" + valid_file_abs + "' to: '" + os.path.join(img_path,"valid",directories[ii]))
            shutil.move(valid_file_abs, os.path.join(img_path_new,"valid",directories[ii]))
        for test_file in test_files:
            test_file_abs = os.path.join(os.getcwd(),test_file)
            #print("move: '" + test_file_abs + "' to: '" + os.path.join(img_path,"test",directories[ii]))
            shutil.move(test_file_abs, os.path.join(img_path_new,"test",directories[ii]))
        
print(directories)
print(per_label_count)
valid_pct = 0.1
test_pct = 0.1       
split_train_valid_test(directories, per_label_count, valid_pct, test_pct)

['Consolidation', 'Hernia', 'No_Finding', 'Nodule', 'Atelectasis', 'Pneumothorax', 'Pleural_Thickening', 'Infiltration', 'Fibrosis', 'Pneumonia', 'Emphysema', 'Mass', 'Effusion', 'Edema', 'Cardiomegaly']
[4667, 227, 60361, 6331, 11559, 5302, 3385, 19894, 1686, 1431, 2516, 5782, 13317, 2303, 2776]


In [10]:
### Data Generetor

train_datagen = ImageDataGenerator(
    #rescale=1./255,
    #shear_range=0.2,
    #zoom_range=0.2,
    #horizontal_flip=True, 
    preprocessing_function=preprocess_input
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

batch_size = 32

train_generator = train_datagen.flow_from_directory(
    '/home/Erdal.Genc/covid_work/deneme/train',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')
    #color_mode='grayscale')

validation_generator = test_datagen.flow_from_directory(
    '/home/Erdal.Genc/covid_work/deneme/valid',
    target_size=(img_height, img_width),
    batch_size=batch_size, #val_batch_size,
    class_mode='categorical')
    
test_generator = test_datagen.flow_from_directory(
    '/home/Erdal.Genc/covid_work/deneme/test',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 113243 images belonging to 15 classes.
Found 14147 images belonging to 15 classes.
Found 14147 images belonging to 15 classes.


In [11]:
def identity_block(X, f, filters, stage, block):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value. We'll need this later to add back to the main path. 
    X_shortcut = X

    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [12]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    
    ##### SHORTCUT PATH ####
    X_shortcut = Conv2D(F3, (1, 1), strides = (s,s), name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [13]:
def ResNet50(input_shape = (64, 64, 3), classes = 14):   
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    #X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    #X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    #X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    #X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    #X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    #X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    #X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    #X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL.
    X = AveragePooling2D((2, 2), name='avg_pool')(X)

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [16]:
model= ResNet50(input_shape=(img_height, img_width,3), classes= 16)
model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [ ]:

nb_train_samples = 113245 #8094 #3036 #18046 #111589 #113243 #139987 
nb_validation_samples= 14147 
epochs = int(nb_train_samples/batch_size)*3
history = model.fit_generator(
    train_generator,
    steps_per_epoch=batch_size, #nb_train_samples/batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=batch_size, #nb_validation_samples/batch_size, #val_batch_size,
    verbose=1)

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 32 steps, validate for 32 steps
Epoch 1/10614
